In [4]:
import numpy as np
import pandas as pd
import re
from BondGraphTools import *
from network_modelling import *
from network_visualization import *

from bokeh.io import show, output_notebook

from os import environ
output_notebook()

Loading BokehJS ...

In [5]:
user = environ["USERNAME"]
#from work
#datafolder = 'C:/Users/DavideMariani/Tradeteq Dropbox/Davide Mariani/thesis_project/'

#from home
datafolder = "C:/Users/{}/Dropbox/University/MscDataScience_Birkbeck/thesis_project/data/".format(user)
inst = pd.read_pickle(datafolder+'snapshots_cd-d-cstats.pkl')

In [6]:
#selecting one snapshot
snapshot = 'sshot_35_'
s35 = inst[inst[snapshot]][[c for c in inst.columns if snapshot in c] \
                      +['has_purchase', 'due_date', 'invoice_date', 'customer_name_1', 
                       'debtor_name_1', 'customer_id', 'debtor_id', 'uid']]
s35 = s35[s35.has_purchase]
#s35.transpose()

In [7]:
s35.shape

(3875, 53)

In [8]:
#max number of pastdues instruments between a customer and a debtor at snapshot 35
s35.groupby(['customer_name_1', 'debtor_name_1']).sshot_35_cd_pastdue90_c.max()

customer_name_1          debtor_name_1                                
ABC Steel Products GBP   Amerex Ltd.                                      0.0
                         Atlantic Steel Processing Ltd                    0.0
                         CJ Upton & Sons Ltd                              0.0
                         Chainbridge Steel (North East) Ltd               0.0
                         Cutler & Woolf Steel Ltd                         0.0
                         D.N.T Company Ltd                                0.0
                         Davro Steel Ltd                                  0.0
                         Fowle & Co Ltd                                   0.0
                         Hi-Tech Steel Services Ltd                       0.0
                         JCY Steel Supplies Ltd                           0.0
                         South Essex Stockholders Ltd                     0.0
                         Top Tubes Ltd                                 

In [9]:
s35.columns

Index(['sshot_35_', 'sshot_35_payment_date', 'sshot_35_payment_amount',
       'sshot_35_last_payment_amount', 'sshot_35_last_payment_date',
       'sshot_35_total_repayment', 'sshot_35_is_pastdue90',
       'sshot_35_is_pastdue180', 'sshot_35_payment_date_mismatch',
       'sshot_35_is_open', 'sshot_35_we_payment_share', 'sshot_35_is_due',
       'sshot_35_cd_lent_c', 'sshot_35_cd_repaid_c', 'sshot_35_cd_pastdue90_c',
       'sshot_35_cd_pastdue180_c', 'sshot_35_cd_trend_a',
       'sshot_35_cd_we_payment_share', 'sshot_35_cd_pd_mismatch_mean',
       'sshot_35_cd_pd_mismatch_std', 'sshot_35_cd_repaid_r',
       'sshot_35_cd_pastdue90_r', 'sshot_35_cd_pastdue180_r',
       'sshot_35_d_lent_c', 'sshot_35_d_repaid_c', 'sshot_35_d_pastdue90_c',
       'sshot_35_d_pastdue180_c', 'sshot_35_d_trend_a',
       'sshot_35_d_we_payment_share', 'sshot_35_d_pd_mismatch_mean',
       'sshot_35_d_pd_mismatch_std', 'sshot_35_d_repaid_r',
       'sshot_35_d_pastdue90_r', 'sshot_35_d_pastdue180_r',
  

In [10]:
fields = ['sshot_35_cd_lent_c', 'sshot_35_cd_repaid_c', 'sshot_35_cd_pastdue90_c',
       'sshot_35_cd_pastdue180_c', 'sshot_35_cd_trend_a',
       'sshot_35_cd_we_payment_share', 'sshot_35_cd_pd_mismatch_mean',
       'sshot_35_cd_pd_mismatch_std', 'sshot_35_cd_repaid_r',
       'sshot_35_cd_pastdue90_r', 'sshot_35_cd_pastdue180_r',
       'sshot_35_d_lent_c', 'sshot_35_d_repaid_c', 'sshot_35_d_pastdue90_c',
       'sshot_35_d_pastdue180_c', 'sshot_35_d_trend_a',
       'sshot_35_d_we_payment_share', 'sshot_35_d_pd_mismatch_mean',
       'sshot_35_d_pd_mismatch_std', 'sshot_35_d_repaid_r',
       'sshot_35_d_pastdue90_r', 'sshot_35_d_pastdue180_r',
       'sshot_35_c_lent_c', 'sshot_35_c_repaid_c', 'sshot_35_c_pastdue90_c',
       'sshot_35_c_pastdue180_c', 'sshot_35_c_trend_a',
       'sshot_35_c_we_payment_share', 'sshot_35_c_pd_mismatch_mean',
       'sshot_35_c_pd_mismatch_std', 'sshot_35_c_repaid_r',
       'sshot_35_c_pastdue90_r', 'sshot_35_c_pastdue180_r']

In [11]:
s35.sshot_35_d_pd_mismatch_std.max()

60.33517492585344

In [12]:
dtemp = prepare_connections(s35, fields = fields)

In [13]:
dtemp.sshot_35_d_pd_mismatch_std.max()

60.33517492585344

In [14]:
dtemp

,customer_id,uid,customer_name_1,debtor_id,debtor_name_1,sshot_35_cd_lent_c,sshot_35_cd_repaid_c,sshot_35_cd_pastdue90_c,sshot_35_cd_pastdue180_c,sshot_35_cd_trend_a,...,sshot_35_c_repaid_c_2,sshot_35_c_pastdue90_c_2,sshot_35_c_pastdue180_c_2,sshot_35_c_trend_a_2,sshot_35_c_we_payment_share_2,sshot_35_c_pd_mismatch_mean_2,sshot_35_c_pd_mismatch_std_2,sshot_35_c_repaid_r_2,sshot_35_c_pastdue90_r_2,sshot_35_c_pastdue180_r_2
24,2004016,1112:717/1157,inter personal GmbH,717,Team Fortis GmbH,4.0,0.0,0.0,0.0,-6.717768,...,0.0,0.0,0.0,-5.394082,NaN,NaN,NaN,0.000000,0.000000,0.000000
28,2004004,600188:1121/172,CDP SWISS AG,1121,Dourado Cabral Sensation Daniella Caroline Dou...,1.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000
32,2004008,4325GU:244/228,jobs united GmbH,244,AZ Elektro AG,1.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.439237,NaN,NaN,NaN,0.000000,0.000000,0.000000
36,2004004,600189:1090/174,CDP SWISS AG,1090,Natural Skin GmbH,1.0,0.0,1.0,0.0,0.000000,...,0.0,3.0,2.0,-7.580725,NaN,NaN,NaN,0.000000,1.000000,0.666667
765,che2004001,NM-618-14:1020000/57,Speo Sàrl,1020000,Bertrandt SA,26.0,0.0,0.0,0.0,3.744508,...,0.0,0.0,0.0,-4.231310,NaN,NaN,NaN,0.000000,0.000000,0.000000
829,2004009,3482:699/706,PM Personal GmbH,699,Stöckli Bodenbeläge AG,7.0,0.0,0.0,0.0,-4.869524,...,0.0,0.0,0.0,-2.458218,NaN,NaN,NaN,0.000000,0.000000,0.000000
833,2004014,1123:692/1076,ONJO AG,692,Leu und Partner Elektro AG,1.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000
852,che2004011,14-101:1070029/118,Swiscovital AG,1070029,GEHE Pharma Handel GmbH,18.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000
1470,2004005,15003:3504/196,PAMAG Personal St. Gallen GmbH,3504,ZKD Büromöbel AG,12.0,0.0,0.0,0.0,6.131416,...,0.0,0.0,0.0,6.131416,NaN,NaN,NaN,0.000000,0.000000,0.000000
1506,2004011,15-179:1040009/837,Creative Promotions GmbH (CHF02),1040009,Diageo Singapore Pte. Ltd.,5.0,0.0,2.0,0.0,-8.582284,...,0.0,2.0,0.0,-8.582284,NaN,NaN,NaN,0.000000,0.400000,0.000000


In [15]:
nodes = create_nodes_df(dtemp)

In [16]:
nodes.head()

,Company_Name,ID,Type,Type_2
0,inter personal GmbH,'2004016',seller,seller
1,CDP SWISS AG,'2004004',seller,seller
2,jobs united GmbH,'2004008',seller,seller
3,Speo Sàrl,'che2004001',seller,seller
4,PM Personal GmbH,'2004009',seller,seller


In [17]:
s35.columns

Index(['sshot_35_', 'sshot_35_payment_date', 'sshot_35_payment_amount',
       'sshot_35_last_payment_amount', 'sshot_35_last_payment_date',
       'sshot_35_total_repayment', 'sshot_35_is_pastdue90',
       'sshot_35_is_pastdue180', 'sshot_35_payment_date_mismatch',
       'sshot_35_is_open', 'sshot_35_we_payment_share', 'sshot_35_is_due',
       'sshot_35_cd_lent_c', 'sshot_35_cd_repaid_c', 'sshot_35_cd_pastdue90_c',
       'sshot_35_cd_pastdue180_c', 'sshot_35_cd_trend_a',
       'sshot_35_cd_we_payment_share', 'sshot_35_cd_pd_mismatch_mean',
       'sshot_35_cd_pd_mismatch_std', 'sshot_35_cd_repaid_r',
       'sshot_35_cd_pastdue90_r', 'sshot_35_cd_pastdue180_r',
       'sshot_35_d_lent_c', 'sshot_35_d_repaid_c', 'sshot_35_d_pastdue90_c',
       'sshot_35_d_pastdue180_c', 'sshot_35_d_trend_a',
       'sshot_35_d_we_payment_share', 'sshot_35_d_pd_mismatch_mean',
       'sshot_35_d_pd_mismatch_std', 'sshot_35_d_repaid_r',
       'sshot_35_d_pastdue90_r', 'sshot_35_d_pastdue180_r',
  

In [19]:
edges = create_edges_df(dtemp, fields = ['sshot_35_cd_pastdue180_c'])

In [20]:
edges[edges.sshot_35_cd_pastdue180_c==4]

,xs,ys,edges_couples,sshot_35_cd_pastdue180_c
Fortuna Personalberatung GmbH,Fortuna Personalberatung GmbH,Baugeschäft Gysi AG,"(Fortuna Personalberatung GmbH, Baugeschäft Gy...",4.0


In [21]:
n, edges, nodes = create_network(edges, nodes)

C:\Users\david\Dropbox\University\MscDataScience_Birkbeck\thesis_project\code\network_modelling.py:182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  nodes['centrality'] = [centrality[n] for n in list(nodes['Company_Name'])]
C:\Users\david\Dropbox\University\MscDataScience_Birkbeck\thesis_project\code\network_modelling.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  nodes['size'] = np.around(np.interp(nodes['centrality'], (nodes['centrality'].min(), nodes['centrality'].max()), (min_size, max_size)),2)


In [22]:
nodes.head()

,Company_Name,ID,Type,Type_2,centrality,size
0,inter personal GmbH,'2004016',seller,seller,0.002538,6.00
29,Team Fortis GmbH,'717',buyer,buyer,0.002538,6.00
1,CDP SWISS AG,'2004004',seller,seller,0.005076,6.09
30,Dourado Cabral Sensation Daniella Caroline Dou...,'1121',buyer,buyer,0.002538,6.00
2,jobs united GmbH,'2004008',seller,seller,0.002538,6.00


In [23]:
g = visualize_graph(n, edges, nodes, nx_k=0.08)

In [24]:
show(g)

## Bondgraphtool experiment

In [21]:
model = new(name='TTQ')

In [22]:
customer1 = new()